# 1-Canny edge detection
## gaussian and first derivative based
#### gray->blur->gradiet(sobel)->non maximum suppression(using gradient magnitude and gradient direction)->double thresholding   ->hysteresis thresholding




In [177]:
import cv2 as cv

In [179]:
import numpy as np

In [181]:
def non_max_suppression(gradient_magnitude, gradient_direction):
    M, N = gradient_magnitude.shape
    result = np.zeros_like(gradient_magnitude)
    for i in range(1, M-1):
        for j in range(1, N-1):
            direction = gradient_direction[i, j] * 180 / np.pi
            if 0 <= direction < 22.5 or 157.5 <= direction <= 180:
                neighbor1, neighbor2 = gradient_magnitude[i, j+1], gradient_magnitude[i, j-1]
            elif 22.5 <= direction < 67.5:
                neighbor1, neighbor2 = gradient_magnitude[i+1, j+1], gradient_magnitude[i-1, j-1]
            elif 67.5 <= direction < 112.5:
                neighbor1, neighbor2 = gradient_magnitude[i+1, j], gradient_magnitude[i-1, j]
            else:
                neighbor1, neighbor2 = gradient_magnitude[i+1, j-1], gradient_magnitude[i-1, j+1]
            if gradient_magnitude[i, j] >= neighbor1 and gradient_magnitude[i, j] >= neighbor2:
                result[i, j] = gradient_magnitude[i, j]
    return result

# we look for nearby pixels in a pixels kernel(3 by 3) in the direction of gradient and if the next pixel has higher intensity then it makes itself 0(black) else retains

In [183]:
def double_thresholding(image, high_threshold, low_threshold):
    result = np.zeros_like(image)
    strong_edges = image > high_threshold
    weak_edges = (image > low_threshold) & (image <= high_threshold)
    result[strong_edges] = 255
    result[weak_edges] = 128
    return result

#sets strong and weak edges for a given threshold range

In [185]:
def hysteresis_thresholding(image):
    result = np.copy(image)
    for i in range(1, image.shape[0] - 1):
        for j in range(1, image.shape[1] - 1):
            if image[i, j] == 128:
                if (image[i+1, j] == 255) or (image[i-1, j] == 255) or (image[i, j+1] == 255) or (image[i, j-1] == 255) or (image[i+1, j+1] == 255) or (image[i-1, j-1] == 255) or (image[i+1, j-1] == 255) or (image[i-1, j+1] == 255):
                    result[i, j] = 255
                else:
                    result[i, j] = 0
    return result

# weak edges are retained only when they are connected to strong edges otherwise set to 0

In [187]:
img=cv.imread(r"C:\Users\chinm\Desktop\sem_3\ieeeMl\jupNb\resources\car.jpg")

In [189]:
gray=cv.cvtColor(img,cv.COLOR_BGR2GRAY)

In [191]:
gauss=cv.GaussianBlur(gray,(3,5),0)

In [195]:
sobelx=cv.Sobel(gauss,cv.CV_64F,1,0)
sobely=cv.Sobel(gauss,cv.CV_64F,0,1)

In [199]:
gradient_magnitude = np.sqrt(sobelx**2 + sobely**2)
gradient_direction = np.arctan2(sobely, sobelx)

In [201]:
non_max_suppressed = non_max_suppression(gradient_magnitude, gradient_direction)

In [203]:
thresholded_image = double_thresholding(non_max_suppressed, 200, 50)

In [205]:
hysteresis_threshold=hysteresis_thresholding(thresholded_image)

In [207]:
canny =cv.Canny(img,190,220)
cv.imshow('hysteresis_threshold',hysteresis_threshold)
cv.imshow('canny',canny)
cv.waitKey(0)

-1

# 2-Sobel Edge Detection
## second derivative and gaussian based
#### blurredAndGrayscaledImage->define sobel kernels->apply convolution to get gradient->thresholding

In [209]:
def sobel_edge_detection(image, threshold):
    kernel_x = [[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]]
    kernel_y = [[1, 2, 1], [0, 0, 0], [-1, -2, -1]]
    edge_map=np.zeros_like(image)
    for y in range(1, len(image) - 1):
        for x in range(1, len(image[0]) - 1):
            gx = (kernel_x[0][0] * image[y - 1][x - 1]) + \
                 (kernel_x[0][1] * image[y - 1][x]) + \
                 (kernel_x[0][2] * image[y - 1][x + 1]) + \
                 (kernel_x[1][0] * image[y][x - 1]) + \
                 (kernel_x[1][1] * image[y][x]) + \
                 (kernel_x[1][2] * image[y][x + 1]) + \
                 (kernel_x[2][0] * image[y + 1][x - 1]) + \
                 (kernel_x[2][1] * image[y + 1][x]) + \
                 (kernel_x[2][2] * image[y + 1][x + 1])
            gy = (kernel_y[0][0] * image[y - 1][x - 1]) + \
                 (kernel_y[0][1] * image[y - 1][x]) + \
                 (kernel_y[0][2] * image[y - 1][x + 1]) + \
                 (kernel_y[1][0] * image[y][x - 1]) + \
                 (kernel_y[1][1] * image[y][x]) + \
                 (kernel_y[1][2] * image[y][x + 1]) + \
                 (kernel_y[2][0] * image[y + 1][x - 1]) + \
                 (kernel_y[2][1] * image[y + 1][x]) + \
                 (kernel_y[2][2] * image[y + 1][x + 1])
            magnitude = (gx**2 + gy**2)**0.5
            edge_map[y][x] = 255 if magnitude > threshold else 0
    edge_map = np.array(edge_map, dtype=np.uint8)
    return edge_map

In [211]:
image=sobel_edge_detection(gauss,150)
cv.imshow('sobel',image)
cv.waitKey(0)

-1